Follow the official Pytorch demo tutorial for Transformer Translation: Transformer model for language understandingLinks to an external site.



Step1. Run the demo and train a model on the original German-to-English training set.

Step2. Train a New model of the same architecture on the opposite training set (English-to-German)

Step3. Insert novel sentences into your English-to-German model. Take the output and feed it to the original German-to-English model. Observe and report qualitatively on the results

## Step1. Run the demo and train a model on the original German-to-English training set.

In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

In [2]:
!pip install -U torchdata
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm
!pip install portalocker>=2.0.0
!pip install torchtext

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [22]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List


# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE_de_to_en = 'de'
TGT_LANGUAGE_de_to_en = 'en'

# Place-holders
token_transform_de_to_en = {}
vocab_transform_de_to_en = {}

In [23]:
token_transform_de_to_en[SRC_LANGUAGE_de_to_en] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform_de_to_en[TGT_LANGUAGE_de_to_en] = get_tokenizer('spacy', language='en_core_web_sm')


# helper function to yield list of tokens
def yield_tokens_de_to_en(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE_de_to_en: 0, TGT_LANGUAGE_de_to_en: 1}

    for data_sample in data_iter:
        yield token_transform_de_to_en[language](data_sample[language_index[language]])

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

"""
Purpose: Defines special tokens used in the vocabulary for machine learning tasks with text data.
Special Tokens:
<unk>: "Unknown" token (represents words not in the vocabulary)
<pad>: Padding token (to make sequences the same length)
<bos>: "Beginning of Sequence"
<eos>: "End of Sequence"
"""

for ln in [SRC_LANGUAGE_de_to_en, TGT_LANGUAGE_de_to_en]:
    # Training data Iterator
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE_de_to_en, TGT_LANGUAGE_de_to_en))
    # Create torchtext's Vocab object
    vocab_transform_de_to_en[ln] = build_vocab_from_iterator(yield_tokens_de_to_en(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE_de_to_en, TGT_LANGUAGE_de_to_en]:
  vocab_transform_de_to_en[ln].set_default_index(UNK_IDX)

In [24]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding_de_to_en(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding_de_to_en, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding_de_to_en(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding_de_to_en, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer_de_to_en(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer_de_to_en, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding_de_to_en(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding_de_to_en(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding_de_to_en(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [25]:
def generate_square_subsequent_mask_de_to_en(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask_de_to_en(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask_de_to_en(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [26]:
torch.manual_seed(0)

SRC_VOCAB_SIZE_de_to_en = len(vocab_transform_de_to_en[SRC_LANGUAGE_de_to_en])
TGT_VOCAB_SIZE_de_to_en = len(vocab_transform_de_to_en[TGT_LANGUAGE_de_to_en])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer_de_to_en = Seq2SeqTransformer_de_to_en(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE_de_to_en, TGT_VOCAB_SIZE_de_to_en, FFN_HID_DIM)

for p in transformer_de_to_en.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer_de_to_en = transformer_de_to_en.to(DEVICE)

loss_fn_de_to_en = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer_de_to_en = torch.optim.Adam(transformer_de_to_en.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [27]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms_de_to_en(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform_de_to_en(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform_de_to_en = {}
for ln in [SRC_LANGUAGE_de_to_en, TGT_LANGUAGE_de_to_en]:
    text_transform_de_to_en[ln] = sequential_transforms_de_to_en(token_transform_de_to_en[ln], #Tokenization
                                               vocab_transform_de_to_en[ln], #Numericalization
                                               tensor_transform_de_to_en) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn_de_to_en(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform_de_to_en[SRC_LANGUAGE_de_to_en](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform_de_to_en[TGT_LANGUAGE_de_to_en](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [28]:
from torch.utils.data import DataLoader

def train_epoch_de_to_en(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE_de_to_en, TGT_LANGUAGE_de_to_en))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn_de_to_en)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask_de_to_en(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn_de_to_en(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate_de_to_en(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE_de_to_en, TGT_LANGUAGE_de_to_en))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn_de_to_en)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask_de_to_en(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn_de_to_en(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [29]:
from timeit import default_timer as timer
NUM_EPOCHS = 20

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch_de_to_en(transformer_de_to_en, optimizer_de_to_en)
    end_time = timer()
    val_loss = evaluate_de_to_en(transformer_de_to_en)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


# function to generate output sequence using greedy algorithm
def greedy_decode_de_to_en(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask_de_to_en(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate_de_to_en(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform_de_to_en[SRC_LANGUAGE_de_to_en](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode_de_to_en(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform_de_to_en[TGT_LANGUAGE_de_to_en].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

Epoch: 1, Train loss: 5.344, Val loss: 4.107, Epoch time = 43.109s
Epoch: 2, Train loss: 3.760, Val loss: 3.309, Epoch time = 44.983s
Epoch: 3, Train loss: 3.157, Val loss: 2.886, Epoch time = 45.793s
Epoch: 4, Train loss: 2.767, Val loss: 2.639, Epoch time = 45.631s
Epoch: 5, Train loss: 2.477, Val loss: 2.439, Epoch time = 45.210s
Epoch: 6, Train loss: 2.247, Val loss: 2.305, Epoch time = 44.958s
Epoch: 7, Train loss: 2.055, Val loss: 2.208, Epoch time = 44.873s
Epoch: 8, Train loss: 1.893, Val loss: 2.114, Epoch time = 45.269s
Epoch: 9, Train loss: 1.754, Val loss: 2.053, Epoch time = 45.193s
Epoch: 10, Train loss: 1.628, Val loss: 2.007, Epoch time = 45.221s
Epoch: 11, Train loss: 1.519, Val loss: 1.961, Epoch time = 44.307s
Epoch: 12, Train loss: 1.419, Val loss: 1.955, Epoch time = 44.665s
Epoch: 13, Train loss: 1.330, Val loss: 1.969, Epoch time = 45.089s
Epoch: 14, Train loss: 1.245, Val loss: 1.973, Epoch time = 45.389s
Epoch: 15, Train loss: 1.173, Val loss: 1.931, Epoch time

In [30]:
print(translate_de_to_en(transformer_de_to_en, "Eine Gruppe von Menschen steht vor einem Iglu ."))

 A group of people standing in front of an igloo 


## Step2. Train a New model of the same architecture on the opposite training set (English-to-German)

## English to German

In [31]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List


# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE_en_to_de = 'en'
TGT_LANGUAGE_en_to_de = 'de'

# Place-holders
token_transform_en_to_de = {}
vocab_transform_en_to_de = {}

In [32]:
token_transform_en_to_de[SRC_LANGUAGE_en_to_de] = get_tokenizer('spacy', language='en_core_web_sm')
token_transform_en_to_de[TGT_LANGUAGE_en_to_de] = get_tokenizer('spacy', language='de_core_news_sm')


# helper function to yield list of tokens
def yield_tokens_en_to_de(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE_en_to_de: 0, TGT_LANGUAGE_en_to_de: 1}

    for data_sample in data_iter:
        yield token_transform_en_to_de[language](data_sample[language_index[language]])

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

"""
Purpose: Defines special tokens used in the vocabulary for machine learning tasks with text data.
Special Tokens:
<unk>: "Unknown" token (represents words not in the vocabulary)
<pad>: Padding token (to make sequences the same length)
<bos>: "Beginning of Sequence"
<eos>: "End of Sequence"
"""

for ln in [SRC_LANGUAGE_en_to_de, TGT_LANGUAGE_en_to_de]:
    # Training data Iterator
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE_en_to_de, TGT_LANGUAGE_en_to_de))
    # Create torchtext's Vocab object
    vocab_transform_en_to_de[ln] = build_vocab_from_iterator(yield_tokens_en_to_de(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE_en_to_de, TGT_LANGUAGE_en_to_de]:
  vocab_transform_en_to_de[ln].set_default_index(UNK_IDX)

In [33]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding_en_to_de(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding_en_to_de, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding_en_to_de(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding_en_to_de, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer_en_to_de(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer_en_to_de, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding_en_to_de(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding_en_to_de(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding_en_to_de(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [34]:
def generate_square_subsequent_mask_en_to_de(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask_en_to_de(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask_en_to_de(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [35]:
torch.manual_seed(0)

SRC_VOCAB_SIZE_en_to_de = len(vocab_transform_en_to_de[SRC_LANGUAGE_en_to_de])
TGT_VOCAB_SIZE_en_to_de = len(vocab_transform_en_to_de[TGT_LANGUAGE_en_to_de])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer_en_to_de = Seq2SeqTransformer_en_to_de(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE_en_to_de, TGT_VOCAB_SIZE_en_to_de, FFN_HID_DIM)

for p in transformer_en_to_de.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer_en_to_de = transformer_en_to_de.to(DEVICE)

loss_fn_en_to_de = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer_en_to_de = torch.optim.Adam(transformer_en_to_de.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [36]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms_en_to_de(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform_en_to_de(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform_en_to_de = {}
for ln in [SRC_LANGUAGE_en_to_de, TGT_LANGUAGE_en_to_de]:
    text_transform_en_to_de[ln] = sequential_transforms_en_to_de(token_transform_en_to_de[ln], #Tokenization
                                               vocab_transform_en_to_de[ln], #Numericalization
                                               tensor_transform_en_to_de) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn_en_to_de(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform_en_to_de[SRC_LANGUAGE_en_to_de](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform_en_to_de[TGT_LANGUAGE_en_to_de](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [37]:
from torch.utils.data import DataLoader

def train_epoch_en_to_de(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE_en_to_de, TGT_LANGUAGE_en_to_de))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn_en_to_de)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask_en_to_de(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn_en_to_de(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate_en_to_de(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE_en_to_de, TGT_LANGUAGE_en_to_de))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn_en_to_de)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask_en_to_de(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn_en_to_de(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [38]:
from timeit import default_timer as timer
NUM_EPOCHS = 20

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch_en_to_de(transformer_en_to_de, optimizer_en_to_de)
    end_time = timer()
    val_loss = evaluate_en_to_de(transformer_en_to_de)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))


# function to generate output sequence using greedy algorithm
def greedy_decode_en_to_de(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask_en_to_de(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate_en_to_de(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform_en_to_de[SRC_LANGUAGE_en_to_de](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode_en_to_de(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform_en_to_de[TGT_LANGUAGE_en_to_de].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

Epoch: 1, Train loss: 5.903, Val loss: 4.632, Epoch time = 53.228s
Epoch: 2, Train loss: 4.132, Val loss: 3.790, Epoch time = 54.612s
Epoch: 3, Train loss: 3.464, Val loss: 3.328, Epoch time = 53.617s
Epoch: 4, Train loss: 3.026, Val loss: 3.011, Epoch time = 52.989s
Epoch: 5, Train loss: 2.703, Val loss: 2.812, Epoch time = 52.447s
Epoch: 6, Train loss: 2.443, Val loss: 2.648, Epoch time = 52.496s
Epoch: 7, Train loss: 2.230, Val loss: 2.505, Epoch time = 53.107s
Epoch: 8, Train loss: 2.059, Val loss: 2.385, Epoch time = 52.602s
Epoch: 9, Train loss: 1.904, Val loss: 2.293, Epoch time = 53.366s
Epoch: 10, Train loss: 1.767, Val loss: 2.227, Epoch time = 52.395s
Epoch: 11, Train loss: 1.648, Val loss: 2.196, Epoch time = 53.106s
Epoch: 12, Train loss: 1.545, Val loss: 2.179, Epoch time = 52.547s
Epoch: 13, Train loss: 1.452, Val loss: 2.175, Epoch time = 52.573s
Epoch: 14, Train loss: 1.361, Val loss: 2.154, Epoch time = 52.861s
Epoch: 15, Train loss: 1.283, Val loss: 2.076, Epoch time

## Step3. Insert novel sentences into your English-to-German model. Take the output and feed it to the original German-to-English model. Observe and report qualitatively on the results

In [39]:
translated_de = translate_en_to_de(transformer_en_to_de, "A group of people standing in front of an igloo")
print("Translated German:", translated_de)

Translated German:  Eine Gruppe von Personen steht vor einem Iglu . 


In [40]:
translated_en = translate_de_to_en(transformer_de_to_en, translated_de)
print("Translated English:", translated_en)

Translated English:  A group of people standing in front of an igloo 


In [43]:
translated_de = translate_en_to_de(transformer_en_to_de, "A group of student are attending class at the university")
print("Translated German:", translated_de)

Translated German:  Eine Gruppe Schüler besucht die Klasse , die sich die Aufmerksamkeit des Bildes . 


In [44]:
translated_en = translate_de_to_en(transformer_de_to_en, translated_de)
print("Translated English:", translated_en)

Translated English:  A group of students attending the attention of the attention of the frame . 


In [45]:
translated_en = translate_de_to_en(transformer_de_to_en, "Die Leute besuchen den Strand von Santa Cruz")
print("Translated English:", translated_en)

Translated English:  The people are attending the beach from Concert . 


In [46]:
translated_de = translate_en_to_de(transformer_en_to_de, translated_en)
print("Translated German:", translated_de)

Translated German:  Die Passagiere helfen der von Concert des Strands . 


While simple sentences are getting properly translated. I see that the translation is not right for all the sentences, particularly complex sentences.